# Requantify NABEC Expression with Y PAR Masked and Unmasked Using FeatureCounts
- **Author(s)** - Frank Grenn
- **Quick Description:** generate two different reference transcriptomes, index them, setup scripts and run. Try to follow parameters used by amppd:   
amp-pd github: https://github.com/amp-pd/amp-pd-workflows/tree/master/rna/rna-quantification  
amp-pd overview: https://amp-pd.org/transcriptomics-data

In [ ]:
import pandas as pd
import os

In [ ]:
WRKDIR = "/PATH/chrY"
NABECDIR = "/PATH/nabec"
OUTDIR = "/PATH/requant/primaryassembly"

fastq_dir = "/PATH/nabec/fastqs"

## 1) Mask Y PARs in Gencode ref genome using [xyalign](https://github.com/SexChrLab/XYalign)
mask the PAR regions (10001 - 2781479 for PAR1 and 56887903 - 57217415 for PAR2 on chr Y)

`Y_PAR.bed` should contain this:
```
chrY    10000   2781479 PAR1
chrY    56887902        57217415        PAR2
```

In [ ]:

print(f"module load samtools \n\
xyalign --PREPARE_REFERENCE --ref /PATH/GRCh38.primary_assembly.genome.fa \
--xx_ref_out {OUTDIR}/xyalign_ref/reference.XXonly.fasta \
--xy_ref_out {OUTDIR}/xyalign_ref/reference.XY.fasta \
--reference_mask {OUTDIR}/../Y_PAR.bed \
--x_chromosome chrX \
--y_chromosome chrY \
--output_dir {OUTDIR}/xyalign_ref")


check if the YPAR was masked by manually checking the fasta files  
if it worked then there should be more nucleotides marked with N's in the reference.XY.fasta

In [ ]:
#check YPAR1 sequences in the original ref genome

#find line number of chrY
print("grep -n chrY /PATH/GRCh38.primary_assembly.genome.fa")
#line 50517410

#find chrY in index
print("grep -n chrY /PATH/GRCh38.primary_assembly.genome.fa.fai")
# ~60 nucleotides per line

#we know YPAR1 starts at 10000
#10000/60 = ~166
#check a few lines earlier: 160
#check 50517410 + 160 = 50517570
print("more +50517570 /PATH/GRCh38.primary_assembly.genome.fa")


In [ ]:
#check YPAR1 sequences in the masked ref genome

#find line number of chrY
print(f"grep -n chrY {OUTDIR}/xyalign_ref/reference.XY.fasta")
#line 50517410

#find chrY in index
print(f"grep -n chrY {OUTDIR}/xyalign_ref/reference.XY.fasta.fai")
# ~60 nucleotides per line

#we know YPAR1 starts at 10000
#10000/60 = ~166
#check a few lines earlier: 160
#check 50517410 + 160 = 50517570
print(f"more +50517570 {OUTDIR}/xyalign_ref/reference.XY.fasta")

check the end of YPAR1 if you want

In [ ]:
#check YPAR1 sequences in the original ref genome


#we know YPAR1 ends at 2781479
#2781479/60 = ~46357
#check a few lines earlier: 46350
#check 50517410 + 46350 = 50563760
print("more +50563760 /PATH/GRCh38.primary_assembly.genome.fa")


In [ ]:
#check YPAR1 sequences in the masked ref genome


#we know YPAR1 ends at 2781479
#2781479/60 = ~46357
#check a few lines earlier: 46350
#check 50517410 + 46350 = 50563760
print(f"more +50563760 {OUTDIR}/xyalign_ref/reference.XY.fasta")

## 2) Generate Y PAR Masked Ref Genome

In [ ]:
!mkdir {OUTDIR}/gencode_star_index_YPAR_masked

In [ ]:
!echo "#!/bin/bash" >> {OUTDIR}/gencode_star_index_YPAR_masked/gencode_star_index_YPAR_masked.sh
!echo "module load STAR" >> {OUTDIR}/gencode_star_index_YPAR_masked/gencode_star_index_YPAR_masked.sh
!echo "STAR --runThreadN 8 --runMode genomeGenerate \
--genomeDir {OUTDIR}/gencode_star_index_YPAR_masked \
--genomeFastaFiles {OUTDIR}/xyalign_ref/reference.XY.fasta \
--sjdbGTFfile /PATH/gencode.v38.primary_assembly.annotation.gtf \
--sjdbOverhang 100" >> {OUTDIR}/gencode_star_index_YPAR_masked/gencode_star_index_YPAR_masked.sh

In [ ]:
#sbatch --mem=50g --cpus-per-task=16 --time=1-0 gencode_star_index_YPAR_masked.sh

## 3) Generate Default Ref Genome

In [ ]:
!mkdir {OUTDIR}/gencode_star_index_default

In [ ]:
!echo "#!/bin/bash" >> {OUTDIR}/gencode_star_index_default/gencode_star_index_default.sh
!echo "module load STAR" >> {OUTDIR}/gencode_star_index_default/gencode_star_index_default.sh
!echo "STAR --runThreadN 8 --runMode genomeGenerate \
--genomeDir {OUTDIR}/gencode_star_index_default \
--genomeFastaFiles /PATH/GRCh38.primary_assembly.genome.fa \
--sjdbGTFfile /PATH/gencode.v38.primary_assembly.annotation.gtf \
--sjdbOverhang 100" >> {OUTDIR}/gencode_star_index_default/gencode_star_index_default.sh

In [ ]:
#sbatch --mem=50g --cpus-per-task=16 --time=1-0 gencode_star_index_default.sh

## 4) Identify samples we want to quantify  
want males only and samples with haplogroups previously identified

In [ ]:
#get the haplogroup data, which should have the sample ids and should all be male
haplos = pd.read_csv(f"{WRKDIR}/output_nabec/nabec_haplos.csv")
haplos['new_id'] = haplos['new_id']+'fctx'
print(haplos.head())

In [ ]:
#get the samples that we have fastqs for
fastqs = os.listdir(f"{NABECDIR}/fastqs")
print(len(fastqs))
samples = list(set([s.replace("_R2.fastq.gz","").replace("_R1.fastq.gz","") for s in fastqs]))
print(len(samples))
print(samples[0:10])

In [ ]:
#combine these to get the samples we want to run through salmon
samples_to_use = set(samples).intersection(set(haplos.new_id))
print(len(samples_to_use))

## 5) Map with STAR 

In [ ]:

!mkdir {OUTDIR}/star_PAR_masked
!mkdir {OUTDIR}/star_default_ref

In [ ]:
swarm = open(f"{OUTDIR}/star_map_nabec.swarm","w")
for s in samples_to_use:
    print(s)
    r1 = f"{fastq_dir}/{s}_R1.fastq.gz"
    r2 = f"{fastq_dir}/{s}_R2.fastq.gz"
    


    if (os.path.isfile(r1) & os.path.isfile(r2)):
        
            
        !mkdir {OUTDIR}/star_PAR_masked/{s}
        out = f"{OUTDIR}/star_PAR_masked/{s}/{s}"
        genomedir = f"{OUTDIR}/gencode_star_index_YPAR_masked"
    
        #swarm.write(f"STAR --genomeDir {genomedir}/ --runThreadN 8 --readFilesIn {r1} {r2} --winAnchorMultimapNmax 100 --outFilterMultimapNmax 100 --outSAMtype BAM Unsorted --outFileNamePrefix {out} --readFilesCommand gunzip -c --outSAMstrandField intronMotif --outSAMunmapped Within --outFilterMismatchNmax 999 --outFilterMismatchNoverLmax 0.1\n")
        swarm.write(f"STAR \
--genomeDir {genomedir}/ \
--runMode alignReads \
--twopassMode Basic \
--outFileNamePrefix {out} \
--readFilesCommand gunzip -c \
--readFilesIn {r1} {r2} \
--outSAMtype BAM Unsorted \
--outFilterType BySJout \
--outFilterMultimapNmax 20 \
--outFilterMismatchNmax 999 \
--outFilterMismatchNoverLmax 0.1 \
--alignIntronMax 1000000 \
--alignMatesGapMax 1000000 \
--alignSJoverhangMin 8 \
--alignSJDBoverhangMin 1 \
--chimOutType Junctions \
--chimSegmentMin 15 \
--chimJunctionOverhangMin 15 \
--runThreadN 16 \
--outSAMstrandField intronMotif \
--outSAMunmapped Within\n")
        
        
        !mkdir {OUTDIR}/star_default_ref/{s}
        out = f"{OUTDIR}/star_default_ref/{s}/{s}"
        genomedir = f"{OUTDIR}/gencode_star_index_default"
    
        #swarm.write(f"STAR --genomeDir {genomedir}/ --runThreadN 8 --readFilesIn {r1} {r2} --winAnchorMultimapNmax 100 --outFilterMultimapNmax 100 --outSAMtype BAM Unsorted --outFileNamePrefix {out} --readFilesCommand gunzip -c --outSAMstrandField intronMotif --outSAMunmapped Within --outFilterMismatchNmax 999 --outFilterMismatchNoverLmax 0.1\n")
        swarm.write(f"STAR \
--genomeDir {genomedir}/ \
--runMode alignReads \
--twopassMode Basic \
--outFileNamePrefix {out} \
--readFilesCommand gunzip -c \
--readFilesIn {r1} {r2} \
--outSAMtype BAM Unsorted \
--outFilterType BySJout \
--outFilterMultimapNmax 20 \
--outFilterMismatchNmax 999 \
--outFilterMismatchNoverLmax 0.1 \
--alignIntronMax 1000000 \
--alignMatesGapMax 1000000 \
--alignSJoverhangMin 8 \
--alignSJDBoverhangMin 1 \
--chimOutType Junctions \
--chimSegmentMin 15 \
--chimJunctionOverhangMin 15 \
--runThreadN 16 \
--outSAMstrandField intronMotif \
--outSAMunmapped Within\n")
        
        

    #break;
    
swarm.close()

In [ ]:
print(f"swarm -g 50 -t 16 --module STAR --time 8:00:00 --dependency afterany:25556129,25556131 -f {OUTDIR}/star_map_nabec.swarm")

## 6) Sort and index BAMs

In [ ]:
swarm = open(f"{OUTDIR}/sort_bams.swarm","w")
for s in samples_to_use:
    print(s)
    
    out = f"{OUTDIR}/star_PAR_masked/{s}/{s}"
    swarm.write(f"samtools sort -o '{out}Aligned.out.sorted.bam' -@ 10 '{out}Aligned.out.bam'\n")
    
    out = f"{OUTDIR}/star_default_ref/{s}/{s}"
    swarm.write(f"samtools sort -o '{out}Aligned.out.sorted.bam' -@ 10 '{out}Aligned.out.bam'\n")
        
        
    #break;
    
swarm.close()

In [ ]:
print(f"swarm -g 50 -t 20 --module samtools --time 8:00:00 --dependency afterany:25556787 -f {OUTDIR}/sort_bams.swarm")

In [ ]:
swarm = open(f"{OUTDIR}/index_bams.swarm","w")
for s in samples_to_use:
    print(s)

    out = f"{OUTDIR}/star_PAR_masked/{s}/{s}"
    swarm.write(f"samtools index -b '{out}Aligned.out.sorted.bam'\n")
        
    out = f"{OUTDIR}/star_default_ref/{s}/{s}"
    swarm.write(f"samtools index -b '{out}Aligned.out.sorted.bam'\n")
        
    #break;
    
swarm.close()

In [ ]:
print(f"swarm -g 50 -t 20 --module samtools --time 8:00:00 --dependency afterany:25557231 -f {OUTDIR}/index_bams.swarm")

## 7) (Optional) Check strandedness of BAM before running featureCounts
needed for the "s" option in featureCounts  
get bed file from UCSC table browser (needs 12 columns)  
interpret output [here](http://rseqc.sourceforge.net/)

In [ ]:
print("module load rseqc\n\
infer_experiment.py -r knownGene_Gencode_V38.bed -i UM5117fctxAligned.out.sorted.bam")

## 8) Run featureCounts

In [ ]:
!mkdir {OUTDIR}/quants_PAR_masked
!mkdir {OUTDIR}/quants_default_ref

In [ ]:
swarm = open(f"{OUTDIR}/run_featureCounts.swarm","w")
for s in samples_to_use:
    print(s)

    genemap = "/PATH/gencode.v38.primary_assembly.annotation.gtf"
    
    star = f"{OUTDIR}/star_PAR_masked/{s}/{s}"
    out = f"{OUTDIR}/quants_PAR_masked/{s}"
    !mkdir {out}
    swarm.write(f'featureCounts \
-T 2 \
-p \
-t exon \
-g gene_id \
-a "{genemap}" \
-s 2 \
-o "{out}/{s}.featureCounts.tsv" \
"{star}Aligned.out.sorted.bam"\n')
        
    star = f"{OUTDIR}/star_default_ref/{s}/{s}"
    out = f"{OUTDIR}/quants_default_ref/{s}"
    !mkdir {out}
    swarm.write(f'featureCounts \
-T 2 \
-p \
-t exon \
-g gene_id \
-a "{genemap}" \
-s 2 \
-o "{out}/{s}.featureCounts.tsv" \
"{star}Aligned.out.sorted.bam"\n')
        
        
        
    #break;
    
swarm.close()

In [ ]:
#print(f"swarm -g 50 -t 20 --module subread --time 8:00:00 --dependency afterany: -f {OUTDIR}/run_featureCounts.swarm")
print(f"swarm -g 50 -t 20 --module subread --time 8:00:00 --dependency afterany:25557426 -f {OUTDIR}/run_featureCounts.swarm")

## 9) Make Count Matrix

In [ ]:
counts = pd.DataFrame()
for s in samples_to_use:
    print(s)
    df = pd.read_table(f"{OUTDIR}/quants_default_ref/{s}/{s}.featureCounts.tsv",sep = "\s+",skiprows = 1)
    
    #print(df.shape)
    #print(df.head())
    df = df.iloc[:,[0,6]]
    df.columns = ["Geneid", s]
    if(len(counts.index)==0):
        counts = df
    else:
        counts = pd.merge(left = counts, right = df, left_on = "Geneid", right_on = "Geneid")
    
    #if(len(counts.columns) >4):
        #break;
counts.to_csv(f"{OUTDIR}/quants_default_ref/quants_default_ref_matrix.csv",index=None)

In [ ]:
counts.shape

In [ ]:
counts.head()